# AutoGluon Tabular - Essential Functionality

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/autogluon/autogluon/blob/master/docs/tutorials/tabular/tabular-essentials.ipynb)
[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/autogluon/autogluon/blob/master/docs/tutorials/tabular/tabular-essentials.ipynb)


Via a simple `fit()` call, AutoGluon can produce highly-accurate models to predict the values in one column of a data table based on the rest of the columns' values. Use AutoGluon with tabular data for both classification and regression problems. This tutorial demonstrates how to use AutoGluon to produce a classification model that predicts whether or not a person's income exceeds $50,000.

## TabularPredictor

To start, import AutoGluon's [TabularPredictor](../../api/autogluon.tabular.TabularPredictor.rst) and [TabularDataset](../../api/autogluon.core.TabularDataset.rst) classes:

In [ ]:
!pip install autogluon.tabular[all]


In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

Load training data from a [CSV file](https://en.wikipedia.org/wiki/Comma-separated_values) into an AutoGluon Dataset object. This object is essentially equivalent to a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) and the same methods can be applied to both.

In [4]:
!mkdir -p data/inc

In [5]:
!wget -O data/inc/train.csv https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv
!wget -O data/inc/test.csv https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv

--2024-04-28 19:12:21--  https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv
Resolving autogluon.s3.amazonaws.com (autogluon.s3.amazonaws.com)... 52.92.162.33, 52.92.237.1, 52.218.244.67, ...
Connecting to autogluon.s3.amazonaws.com (autogluon.s3.amazonaws.com)|52.92.162.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4573758 (4.4M) [text/csv]
Saving to: ‘data/inc/train.csv’

data/inc/train.csv  100%[===================>]   4.36M  9.80MB/s    in 0.4s    

2024-04-28 19:12:22 (9.80 MB/s) - ‘data/inc/train.csv’ saved [4573758/4573758]

--2024-04-28 19:12:22--  https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv
Resolving autogluon.s3.amazonaws.com (autogluon.s3.amazonaws.com)... 52.92.208.161, 52.92.213.25, 52.92.207.113, ...
Connecting to autogluon.s3.amazonaws.com (autogluon.s3.amazonaws.com)|52.92.208.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1143496 (1.1M) [text/csv]
Saving to: ‘data/inc/test.csv’

data/inc/t

In [3]:
train_data = TabularDataset('data/inc/train.csv')

In [4]:
subsample_size = 10000  # subsample subset of data for faster demo, try setting this to much larger values

In [5]:
train_data = train_data.sample(n=subsample_size, random_state=0)
train_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
6118,51,Private,39264,Some-college,10,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,>50K
23204,58,Private,51662,10th,6,Married-civ-spouse,Other-service,Wife,White,Female,0,0,8,United-States,<=50K
29590,40,Private,326310,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,44,United-States,<=50K
18116,37,Private,222450,HS-grad,9,Never-married,Sales,Not-in-family,White,Male,0,2339,40,El-Salvador,<=50K
33964,62,Private,109190,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,15024,0,40,United-States,>50K


Note that we loaded data from a CSV file stored in the cloud. You can also specify a local file-path instead if you have already downloaded the CSV file to your own machine (e.g., using [wget](https://www.gnu.org/software/wget/)).
Each row in the table `train_data` corresponds to a single training example. In this particular dataset, each row corresponds to an individual person, and the columns contain various characteristics reported during a census.

Let's first use these features to predict whether the person's income exceeds $50,000 or not, which is recorded in the `class` column of this table.

In [6]:
label = 'class'
print(f"Unique classes: {list(train_data[label].unique())}")

Unique classes: [' >50K', ' <=50K']


AutoGluon works with raw data, meaning you don't need to perform any data preprocessing before fitting AutoGluon. We actively recommend that you avoid performing operations such as missing value imputation or one-hot-encoding, as AutoGluon has dedicated logic to handle these situations automatically. You can learn more about AutoGluon's preprocessing in the [Feature Engineering Tutorial](tabular-feature-engineering.ipynb).

### Training

Now we initialize and fit AutoGluon's TabularPredictor in one line of code:

In [11]:
predictor = TabularPredictor(label=label).fit(train_data)

No path specified. Models will be saved in: "AutogluonModels/ag-20240428_191738"
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20240428_191738"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.6
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Mar 23 09:49:55 UTC 2024
CPU Count:        

That's it! We now have a TabularPredictor that is able to make predictions on new data.

### Prediction

Next, load separate test data to demonstrate how to make predictions on new examples at inference time:

In [7]:
test_data = TabularDataset('data/inc/test.csv')
test_data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,31,Private,169085,11th,7,Married-civ-spouse,Sales,Wife,White,Female,0,0,20,United-States,<=50K
1,17,Self-emp-not-inc,226203,12th,8,Never-married,Sales,Own-child,White,Male,0,0,45,United-States,<=50K
2,47,Private,54260,Assoc-voc,11,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,1887,60,United-States,>50K
3,21,Private,176262,Some-college,10,Never-married,Exec-managerial,Own-child,White,Female,0,0,30,United-States,<=50K
4,17,Private,241185,12th,8,Never-married,Prof-specialty,Own-child,White,Male,0,0,20,United-States,<=50K


We can now use our trained models to make predictions on the new data:

In [13]:
y_pred = predictor.predict(test_data)
y_pred.head()  # Predictions

0     <=50K
1     <=50K
2      >50K
3     <=50K
4     <=50K
Name: class, dtype: object

In [15]:
y_pred_proba = predictor.predict_proba(test_data)
y_pred_proba.head()  # Prediction Probabilities

,<=50K,>50K
0,0.901691,0.098309
1,0.995551,0.004449
2,0.036577,0.963423
3,0.997173,0.002827
4,0.997289,0.002711


### Evaluation

Next, we can [evaluate](../../api/autogluon.tabular.TabularPredictor.evaluate.rst) the predictor on the (labeled) test data:

In [16]:
predictor.evaluate(test_data)

{'accuracy': 0.8710205752891801,
 'balanced_accuracy': 0.7839382672575002,
 'mcc': 0.6220637525126514,
 'roc_auc': 0.9254112488042383,
 'f1': 0.6946194861851672,
 'precision': 0.7925884955752213,
 'recall': 0.6182053494391717}

We can also [evaluate each model individually](../../api/autogluon.tabular.TabularPredictor.leaderboard.rst):

In [17]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.871021,0.877,accuracy,0.883530,0.168275,4.813978,0.004538,0.000904,0.188562,2,True,14
1,XGBoost,0.870304,0.876,accuracy,0.104581,0.020128,2.118129,0.104581,0.020128,2.118129,1,True,11
2,CatBoost,0.869895,0.862,accuracy,0.032715,0.012178,16.338418,0.032715,0.012178,16.338418,1,True,7
3,LightGBM,0.868666,0.866,accuracy,0.281120,0.025550,1.361796,0.281120,0.025550,1.361796,1,True,4
4,LightGBMXT,0.862831,0.863,accuracy,1.046283,0.094512,4.655111,1.046283,0.094512,4.655111,1,True,3
5,LightGBMLarge,0.862627,0.868,accuracy,0.060372,0.009796,1.392192,0.060372,0.009796,1.392192,1,True,13
6,RandomForestEntr,0.857406,0.859,accuracy,0.886364,0.121880,3.723767,0.886364,0.121880,3.723767,1,True,6
7,RandomForestGini,0.856075,0.860,accuracy,0.642733,0.222036,3.623287,0.642733,0.222036,3.623287,1,True,5
8,NeuralNetTorch,0.847989,0.842,accuracy,0.097338,0.018209,40.864079,0.097338,0.018209,40.864079,1,True,12
9,ExtraTreesEntr,0.847067,0.845,accuracy,0.774410,0.147244,2.507287,0.774410,0.147244,2.507287,1,True,9


### Loading a Trained Predictor

Finally, we can load the predictor in a new session (or new machine) by calling [TabularPredictor.load()](../../api/autogluon.tabular.TabularPredictor.load.rst) and specifying the location of the predictor artifact on disk.

In [ ]:
predictor.path  # The path on disk where the predictor is saved

In [8]:
# Load the predictor by specifying the path it is saved to on disk.
# You can control where it is saved to by setting the `path` parameter during init
predictor = TabularPredictor.load("AutogluonModels/ag-20240428_191738")

Now you're ready to try AutoGluon on your own tabular datasets!
As long as they're stored in a popular format like CSV, you should be able to achieve strong predictive performance with just 2 lines of code:

```
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label=<variable-name>).fit(train_data=<file-name>)
```


**Note:** This simple call to [TabularPredictor.fit()](../../api/autogluon.tabular.TabularPredictor.fit.rst) is intended for your first prototype model. In a subsequent section, we'll demonstrate how to maximize predictive performance by additionally specifying the `presets` parameter to `fit()` and the `eval_metric` parameter to `TabularPredictor()`.

## Description of fit()

Here we discuss what happened during `fit()`.

Since there are only two possible values of the `class` variable, this was a binary classification problem, for which an appropriate performance metric is _accuracy_. AutoGluon automatically infers this as well as the type of each feature (i.e., which columns contain continuous numbers vs. discrete categories). AutoGluon can also automatically handle common issues like missing data and rescaling feature values.

We did not specify separate validation data and so AutoGluon automatically chose a random training/validation split of the data. The data used for validation is separated from the training data and is used to determine the models and hyperparameter-values that produce the best results. Rather than just a single model, AutoGluon trains multiple models and ensembles them together to obtain superior predictive performance.

By default, AutoGluon tries to fit [various types of models](../../api/autogluon.tabular.models.rst) including neural networks and tree ensembles. Each type of model has various hyperparameters, which traditionally, the user would have to specify. AutoGluon automates this process.

AutoGluon automatically and iteratively tests values for hyperparameters to produce the best performance on the validation data. This involves repeatedly training models under different hyperparameter settings and evaluating their performance. This process can be computationally-intensive, so `fit()` parallelizes this process across multiple threads using [Ray](https://www.ray.io/). To control runtimes, you can specify various arguments in `fit()` such as `time_limit` as demonstrated in the subsequent **[In-Depth Tutorial](tabular-indepth.ipynb)**.

We can view what properties AutoGluon automatically inferred about our prediction task:

In [9]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  binary
AutoGluon identified the following types of features:
('category', [])  : 7 | ['workclass', 'education', 'marital-status', 'occupation', 'relationship', ...]
('int', [])       : 6 | ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', ...]
('int', ['bool']) : 1 | ['sex']


AutoGluon correctly recognized our prediction problem to be a **binary classification** task and decided that variables such as `age` should be represented as integers, whereas variables such as `workclass` should be represented as categorical objects. The `feature_metadata` attribute allows you to see the inferred data type of each predictive variable after preprocessing (this is its _raw_ dtype; some features may also be associated with additional _special_ dtypes if produced via feature-engineering, e.g. numerical representations of a datetime/text column).

To transform the data into AutoGluon's internal representation, we can do the following:

In [11]:
test_data.info()

<class 'autogluon.core.dataset.TabularDataset'>
RangeIndex: 9769 entries, 0 to 9768
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             9769 non-null   int64 
 1   workclass       9769 non-null   object
 2   fnlwgt          9769 non-null   int64 
 3   education       9769 non-null   object
 4   education-num   9769 non-null   int64 
 5   marital-status  9769 non-null   object
 6   occupation      9769 non-null   object
 7   relationship    9769 non-null   object
 8   race            9769 non-null   object
 9   sex             9769 non-null   object
 10  capital-gain    9769 non-null   int64 
 11  capital-loss    9769 non-null   int64 
 12  hours-per-week  9769 non-null   int64 
 13  native-country  9769 non-null   object
 14  class           9769 non-null   object
dtypes: int64(6), object(9)
memory usage: 1.1+ MB


In [12]:
test_data_transform = predictor.transform_features(test_data)
test_data_transform.head()

,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hours-per-week,workclass,education,marital-status,occupation,relationship,race,native-country
0,31,169085,7,0,0,0,20,4,1,2,12,5,4,38
1,17,226203,8,1,0,0,45,6,2,4,12,3,4,38
2,47,54260,11,1,0,1887,60,4,8,2,4,0,4,38
3,21,176262,10,0,0,0,30,4,15,4,4,3,4,38
4,17,241185,8,1,0,0,20,4,2,4,10,3,4,38


In [13]:
test_data_transform.info()

<class 'autogluon.core.dataset.TabularDataset'>
RangeIndex: 9769 entries, 0 to 9768
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             9769 non-null   int64   
 1   fnlwgt          9769 non-null   int64   
 2   education-num   9769 non-null   int64   
 3   sex             9769 non-null   int8    
 4   capital-gain    9769 non-null   int64   
 5   capital-loss    9769 non-null   int64   
 6   hours-per-week  9769 non-null   int64   
 7   workclass       9769 non-null   category
 8   education       9769 non-null   category
 9   marital-status  9769 non-null   category
 10  occupation      9769 non-null   category
 11  relationship    9769 non-null   category
 12  race            9769 non-null   category
 13  native-country  9769 non-null   category
dtypes: category(7), int64(6), int8(1)
memory usage: 535.2 KB


Notice how the data is purely numeric after pre-processing (although categorical features will still be treated as categorical downstream).

To better understand our trained predictor, we can estimate the overall importance of each feature via [TabularPredictor.feature_importance()](../../api/autogluon.tabular.TabularPredictor.feature_importance.rst):

In [14]:
predictor.feature_importance(test_data)

,importance,stddev,p_value,n,p99_high,p99_low
marital-status,0.04856,0.002837,0.000001,5,0.054401,0.042719
capital-gain,0.04412,0.003448,0.000004,5,0.051220,0.037020
education-num,0.03000,0.005936,0.000175,5,0.042223,0.017777
capital-loss,0.01052,0.002124,0.000189,5,0.014894,0.006146
age,0.01024,0.002360,0.000316,5,0.015099,0.005381
occupation,0.00984,0.003186,0.001152,5,0.016399,0.003281
hours-per-week,0.00600,0.003787,0.011975,5,0.013797,-0.001797
workclass,0.00236,0.001284,0.007362,5,0.005003,-0.000283
relationship,0.00140,0.000894,0.012448,5,0.003242,-0.000442
sex,0.00064,0.001228,0.154312,5,0.003168,-0.001888


The `importance` column is an estimate for the amount the evaluation metric score would drop if the feature were removed from the data.
Negative values of `importance` mean that it is likely to improve the results if re-fit with the feature removed.

When we call `predict()`, AutoGluon automatically predicts with the model that displayed the best performance on validation data (i.e. the weighted-ensemble).

In [15]:
predictor.model_best

'WeightedEnsemble_L2'

We can instead specify which model to use for predictions like this:

```
predictor.predict(test_data, model='LightGBM')
```

You can get the list of trained models via `.leaderboard()` or `.model_names()`:

In [16]:
predictor.model_names()

['KNeighborsUnif',
 'KNeighborsDist',
 'LightGBMXT',
 'LightGBM',
 'RandomForestGini',
 'RandomForestEntr',
 'CatBoost',
 'ExtraTreesGini',
 'ExtraTreesEntr',
 'NeuralNetFastAI',
 'XGBoost',
 'NeuralNetTorch',
 'LightGBMLarge',
 'WeightedEnsemble_L2']

The scores of predictive performance above were based on a default evaluation metric (accuracy for binary classification). Performance in certain applications may be measured by different metrics than the ones AutoGluon optimizes for by default. If you know the metric that counts in your application, you should specify it via the `eval_metric` argument as demonstrated in the next section.

## Presets

AutoGluon comes with a variety of presets that can be specified in the call to `.fit` via the `presets` argument. `medium_quality` is used by default to encourage initial prototyping, but for serious usage, the other presets should be used instead.

| Preset         | Model Quality                                          | Use Cases                                                                                                                                               | Fit Time (Ideal) | Inference Time (Relative to medium_quality) | Disk Usage |
| :------------- |:-------------------------------------------------------| :------------------------------------------------------------------------------------------------------------------------------------------------------ |:-----------------| :------------------------------------------ | :--------- |
| best_quality   | State-of-the-art (SOTA), much better than high_quality | When accuracy is what matters                                                                                                                           | 16x+             | 32x+                                        | 16x+       |
| high_quality   | Better than good_quality                               | When a very powerful, portable solution with fast inference is required: Large-scale batch inference                                                    | 16x+             | 4x                                          | 2x         |
| good_quality   | Stronger than any other AutoML Framework               | When a powerful, highly portable solution with very fast inference is required: Billion-scale batch inference, sub-100ms online-inference, edge-devices | 16x              | 2x                                          | 0.1x       |
| medium_quality | Competitive with other top AutoML Frameworks           | Initial prototyping, establishing a performance baseline                                                                                                | 1x               | 1x                                          | 1x         |

We recommend users to start with `medium_quality` to get a sense of the problem and identify any data related issues. If `medium_quality` is taking too long to train, consider subsampling the training data during this prototyping phase.  
Once you are comfortable, next try `best_quality`. Make sure to specify at least 16x the `time_limit` value as used in `medium_quality`. Once finished, you should have a very powerful solution that is often stronger than `medium_quality`.  
Make sure to consider holding out test data that AutoGluon never sees during training to ensure that the models are performing as expected in terms of performance.  
Once you evaluate both `best_quality` and `medium_quality`, check if either satisfies your needs. If neither do, consider trying `high_quality` and/or `good_quality`.  
If none of the presets satisfy requirements, refer to [Predicting Columns in a Table - In Depth](tabular-indepth.ipynb) for more advanced AutoGluon options.

## Maximizing predictive performance

**Note:** You should not call `fit()` with entirely default arguments if you are benchmarking AutoGluon-Tabular or hoping to maximize its accuracy!
To get the best predictive accuracy with AutoGluon, you should generally use it like this:

In [17]:
time_limit = 120  # for quick demonstration only, you should set this to longest time you are willing to wait (in seconds)
metric = 'roc_auc'  # specify your evaluation metric here
predictor = TabularPredictor(label, eval_metric=metric).fit(train_data, time_limit=time_limit, presets='best_quality')

No path specified. Models will be saved in: "AutogluonModels/ag-20240428_192731"
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 120 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240428_192731/ds_sub_fit/sub_fit_ho.
2024-04-28 19:27:31,962	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.

In [18]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBM_BAG_L1,0.926662,0.925289,roc_auc,1.512923,0.492747,23.799725,1.512923,0.492747,23.799725,1,True,4
1,WeightedEnsemble_L3,0.926510,0.925415,roc_auc,6.594110,1.068564,50.123345,0.006805,0.002121,0.245950,3,True,7
2,WeightedEnsemble_L2,0.926510,0.925415,roc_auc,6.600583,1.080371,50.092892,0.013278,0.013928,0.215498,2,True,5
3,LightGBMXT_BAG_L2,0.926007,0.881787,roc_auc,6.895697,1.411245,72.341072,0.118196,0.129014,22.408636,2,True,6
4,LightGBMXT_BAG_L1,0.921432,0.920403,roc_auc,5.074381,0.573696,26.077669,5.074381,0.573696,26.077669,1,True,3
5,KNeighborsDist_BAG_L1,0.640920,0.636578,roc_auc,0.079136,0.139756,0.024487,0.079136,0.139756,0.024487,1,True,2
6,KNeighborsUnif_BAG_L1,0.632664,0.625312,roc_auc,0.111060,0.076033,0.030554,0.111060,0.076033,0.030554,1,True,1


This command implements the following strategy to maximize accuracy:

- Specify the argument `presets='best_quality'`, which allows AutoGluon to automatically construct powerful model ensembles based on [stacking/bagging](https://arxiv.org/abs/2003.06505), and will greatly improve the resulting predictions if granted sufficient training time. The default value of `presets` is `'medium_quality'`, which produces _less_ accurate models but facilitates faster prototyping. With `presets`, you can flexibly prioritize predictive accuracy vs. training/inference speed. For example, if you care less about predictive performance and want to quickly deploy a basic model, consider using: `presets=['good_quality', 'optimize_for_deployment']`.

- Provide the parameter `eval_metric` to `TabularPredictor()` if you know what metric will be used to evaluate predictions in your application. Some other non-default metrics you might use include things like: `'f1'` (for binary classification), `'roc_auc'` (for binary classification), `'log_loss'` (for classification), `'mean_absolute_error'` (for regression), `'median_absolute_error'` (for regression). You can also define your own custom metric function. For more information refer to [Adding a custom metric to AutoGluon](advanced/tabular-custom-metric.ipynb).

- Include all your data in `train_data` and do not provide `tuning_data` (AutoGluon will split the data more intelligently to fit its needs).

- Do not specify the `hyperparameter_tune_kwargs` argument (counterintuitively, hyperparameter tuning is not the best way to spend a limited training time budgets, as model ensembling is often superior). We recommend you only use `hyperparameter_tune_kwargs` if your goal is to deploy a single model rather than an ensemble.

- Do not specify the `hyperparameters` argument (allow AutoGluon to adaptively select which models/hyperparameters to use).

- Set `time_limit` to the longest amount of time (in seconds) that you are willing to wait. AutoGluon's predictive performance improves the longer `fit()` is allowed to run.

## Regression (predicting numeric table columns):

To demonstrate that `fit()` can also automatically handle regression tasks, we now try to predict the numeric `age` variable in the same table based on the other features:

In [19]:
age_column = 'age'
train_data[age_column].head()

6118     51
23204    58
29590    40
18116    37
33964    62
Name: age, dtype: int64

We again call `fit()`, imposing a time-limit this time (in seconds), and also demonstrate a shorthand method to evaluate the resulting model on the test data (which contain labels):

In [21]:
predictor_age = TabularPredictor(
    label=age_column, 
    problem_type="regression", 
    path="agModels-predictAge"
).fit(
    train_data, 
    time_limit=60
)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "agModels-predictAge"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.10.6
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Mar 23 09:49:55 UTC 2024
CPU Count:          2
Memory Avail:       2.16 GB / 3.78 GB (57.1%)
Disk Space Avail:   858993458

In [22]:
predictor_age.evaluate(test_data)

{'root_mean_squared_error': -9.918412618392736,
 'mean_squared_error': -98.37490886869226,
 'mean_absolute_error': -7.661268687611874,
 'r2': 0.4741670155829514,
 'pearsonr': 0.6902491589199665,
 'median_absolute_error': -6.188911437988281}

Note that we didn't need to tell AutoGluon this is a regression problem, it automatically inferred this from the data and reported the appropriate performance metric (RMSE by default). To specify a particular evaluation metric other than the default, set the `eval_metric` parameter of [TabularPredictor()](../../api/autogluon.tabular.TabularPredictor.rst) and AutoGluon will tailor its models to optimize your metric (e.g. `eval_metric = 'mean_absolute_error'`). For evaluation metrics where higher values are worse (like RMSE), AutoGluon will flip their sign and print them as negative values during training (as it internally assumes higher values are better). You can even specify a custom metric by following the [Custom Metric Tutorial](advanced/tabular-custom-metric.ipynb).

We can call leaderboard to see the per-model performance:

In [23]:
predictor_age.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,-9.918413,-9.944643,root_mean_squared_error,1.895314,0.357470,39.296275,0.008402,0.000581,0.021895,2,True,10
1,CatBoost,-9.931876,-10.101496,root_mean_squared_error,0.087618,0.017887,28.197619,0.087618,0.017887,28.197619,1,True,6
2,LightGBM,-9.974593,-10.143751,root_mean_squared_error,0.180695,0.020087,1.311310,0.180695,0.020087,1.311310,1,True,4
3,XGBoost,-9.984771,-10.065653,root_mean_squared_error,0.092642,0.016012,0.776343,0.092642,0.016012,0.776343,1,True,9
4,LightGBMXT,-9.995518,-10.119169,root_mean_squared_error,0.164085,0.017478,1.390053,0.164085,0.017478,1.390053,1,True,3
5,RandomForestMSE,-10.460632,-10.427634,root_mean_squared_error,1.404439,0.260369,13.282043,1.404439,0.260369,13.282043,1,True,5
6,ExtraTreesMSE,-10.528087,-10.392588,root_mean_squared_error,1.068797,0.248071,7.530094,1.068797,0.248071,7.530094,1,True,7
7,NeuralNetFastAI,-10.587627,-10.741214,root_mean_squared_error,0.391880,0.041436,1.362239,0.391880,0.041436,1.362239,1,True,8
8,KNeighborsUnif,-14.575427,-14.522793,root_mean_squared_error,0.108898,0.039951,0.045208,0.108898,0.039951,0.045208,1,True,1
9,KNeighborsDist,-14.629805,-14.580066,root_mean_squared_error,0.081890,0.016004,0.018032,0.081890,0.016004,0.018032,1,True,2


**Data Formats:** AutoGluon can currently operate on data tables already loaded into Python as pandas DataFrames, or those stored in files of [CSV format](https://en.wikipedia.org/wiki/Comma-separated_values) or [Parquet format](https://databricks.com/glossary/what-is-parquet). If your data lives in multiple tables, you will first need to join them into a single table whose rows correspond to statistically independent observations (datapoints) and columns correspond to different features (aka. variables/covariates).

Refer to the [TabularPredictor documentation](../../api/autogluon.tabular.TabularPredictor.rst) to see all of the available methods/options.

## Advanced Usage

For more advanced usage examples of AutoGluon, refer to the [In Depth Tutorial](tabular-indepth.ipynb)

If you are interested in deployment optimization, refer to the [Deployment Optimization Tutorial](advanced/tabular-deployment.ipynb).

For adding custom models to AutoGluon, refer to the [Custom Model](advanced/tabular-custom-model.ipynb) and [Custom Model Advanced](advanced/tabular-custom-model-advanced.ipynb) tutorials.